In [44]:
import pandas as pd
import numpy as np
import os
import zipfile
import csv

In [45]:
df = pd.read_csv('metaData.csv')
df.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15


In [34]:
df.groupby('busRoute').size()

busRoute
-      11
31     17
32     26
33    155
46    123
72    158
83    846
N1     16
N2     39
N4     18
dtype: int64

In [5]:
df.dtypes

name                             object
busNumber                         int64
startTime_iso                    object
startTime_unix                    int64
endTime_iso                      object
endTime_unix                      int64
drivenDistance                  float64
busRoute                         object
energyConsumption               float64
itcs_numberOfPassengers_mean    float64
itcs_numberOfPassengers_min       int64
itcs_numberOfPassengers_max       int64
status_gridIsAvailable_mean     float64
temperature_ambient_mean        float64
temperature_ambient_min         float64
temperature_ambient_max         float64
dtype: object

In [46]:
df.tail()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.80899,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.87940,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.10526,0,74,0.495739,279.4540,277.15,291.1500
1408,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10T23:34:46Z,1670715286,2022-12-11T03:29:05Z,1670729345,72293.83,N2,5.350956e+08,15.99320,0,52,0.459492,276.0036,273.15,291.1500


### 1. Extract all trips with `busRoute` 83

In [47]:
df['busRoute'] = df['busRoute'].replace("-", np.nan)
df['busRoute'] = pd.to_numeric(df['busRoute'], errors='coerce')

In [48]:
df.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,NaN,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31.0,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33.0,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,NaN,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72.0,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15


In [49]:
df.tail()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32.0,4.260419e+08,39.80899,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83.0,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83.0,1.415700e+09,29.87940,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,NaN,4.519165e+08,20.10526,0,74,0.495739,279.4540,277.15,291.1500
1408,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10T23:34:46Z,1670715286,2022-12-11T03:29:05Z,1670729345,72293.83,NaN,5.350956e+08,15.99320,0,52,0.459492,276.0036,273.15,291.1500


In [8]:
df[df['busRoute']==83]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,1583210558,2020-03-03T19:44:51Z,1583264691,225047.90,83.0,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,1583470403,2020-03-06T19:44:42Z,1583523882,224512.30,83.0,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,1583763373,2020-03-09T19:34:17Z,1583782457,77824.36,83.0,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,1583815803,2020-03-10T19:51:25Z,1583869885,225095.80,83.0,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,1583989001,2020-03-12T19:44:57Z,1584042297,224181.20,83.0,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,1669783673,2022-11-30T19:50:22Z,1669837822,223165.00,83.0,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,1669871981,2022-12-01T18:20:57Z,1669918857,190196.00,83.0,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,1669956468,2022-12-02T19:40:01Z,1670010001,224473.40,83.0,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83.0,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


### 2. Extract all trips where `busRoute` is not a number

In [50]:
df[df['busRoute'].isnull()]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,NaN,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,NaN,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10T03:16:11Z,1557458171,2019-05-10T18:51:37Z,1557514297,210577.00,NaN,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.15,293.15
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13T03:10:23Z,1557717023,2019-05-13T23:16:13Z,1557789373,267033.80,NaN,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.15,293.15
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24T02:52:47Z,1558666367,2019-05-24T22:35:11Z,1558737311,263432.60,NaN,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.15,299.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,1666391912,2022-10-22T02:42:21Z,1666406541,78567.16,NaN,4.347766e+08,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,1666478085,2022-10-23T02:29:59Z,1666492199,73427.97,NaN,3.997737e+08,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,1669419316,2022-11-26T03:30:39Z,1669433439,72911.26,NaN,4.475534e+08,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,NaN,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.15


### 3. For each (busNumber, busRoute) pair, determine the number of trips

In [10]:
df.groupby(['busRoute', 'busNumber'], as_index=False).size()

,busRoute,busNumber,size
0,31.0,183,12
1,31.0,208,5
2,32.0,183,12
3,32.0,208,14
4,33.0,183,130
5,33.0,208,25
6,46.0,183,104
7,46.0,208,19
8,72.0,183,114
9,72.0,208,44


### 4. For each trip, compute the ratio between the energy consumption and the average number of passengers

In [11]:
df['ratio'] = df['energyConsumption']/df['itcs_numberOfPassengers_mean']

In [12]:
df.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,NaN,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15,8.640500e+07
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31.0,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15,1.214747e+07
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33.0,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15,7.342794e+07
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,NaN,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15,1.673328e+08
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72.0,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15,2.613190e+07


In [31]:
df.tail()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32.0,4.260419e+08,39.80899,0,83,0.739349,279.6404,278.15,291.1500,1.070215e+07
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83.0,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655,5.338462e+07
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83.0,1.415700e+09,29.87940,0,102,0.439916,279.1724,277.15,292.1500,4.738047e+07
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,NaN,4.519165e+08,20.10526,0,74,0.495739,279.4540,277.15,291.1500,2.247753e+07
1408,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10T23:34:46Z,1670715286,2022-12-11T03:29:05Z,1670729345,72293.83,NaN,5.350956e+08,15.99320,0,52,0.459492,276.0036,273.15,291.1500,3.345769e+07


### 5. For each station (`itcs_stopName`), determine the average number of passengers.

In [13]:
directory = '/Users/andreacardinali/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano-Bicocca/focs/Exam project/Project_cs23/ZTBus_compressed'

In [14]:
csv_files = [f for f in os.listdir(directory)]
csv_files

['B183_2020-11-13_14-52-45_2020-11-13_19-13-45.csv',
 'B183_2020-10-06_04-23-44_2020-10-06_07-33-54.csv',
 'B183_2019-10-03_03-04-42_2019-10-03_18-38-45.csv',
 'B183_2022-05-02_03-02-19_2022-05-02_17-07-49.csv',
 'B183_2021-04-23_03-47-54_2021-04-23_07-48-48.csv',
 'B208_2022-08-15_03-31-51_2022-08-15_12-34-10.csv',
 'B183_2020-07-24_04-01-31_2020-07-24_18-04-39.csv',
 'B183_2022-10-28_13-36-23_2022-10-28_16-37-08.csv',
 'B208_2021-04-21_04-10-07_2021-04-21_18-19-32.csv',
 'B183_2022-07-28_14-27-33_2022-07-28_19-17-23.csv',
 'B183_2021-09-14_04-09-57_2021-09-14_11-46-59.csv',
 'B208_2022-04-01_22-29-37_2022-04-02_02-35-00.csv',
 'B208_2021-10-21_12-57-23_2021-10-21_18-26-47.csv',
 'B208_2022-01-13_05-08-12_2022-01-13_19-18-40.csv',
 'B183_2021-12-20_04-59-40_2021-12-20_08-17-47.csv',
 'B183_2022-06-24_23-09-54_2022-06-25_02-17-26.csv',
 'B208_2021-07-29_04-10-00_2021-07-29_17-55-34.csv',
 'B183_2019-07-29_03-07-49_2019-07-29_17-48-21.csv',
 'B208_2022-12-09_23-55-12_2022-12-10_03-24-28

In [15]:
precise = pd.read_csv('/Users/andreacardinali/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano-Bicocca/focs/Exam project/Project_cs23/ZTBus_compressed/B183_2021-05-18_04-23-09_2021-05-18_07-33-23.csv')
precise.groupby('itcs_stopName')['itcs_numberOfPassengers'].mean()

itcs_stopName
-                                   NaN
Zürich, Albisrank             13.333333
Zürich, Albisriederplatz      23.500000
Zürich, Altes Krematorium     21.166667
Zürich, Bahnhof Altstetten     6.000000
Zürich, Bahnhof Hardbrücke    26.833333
Zürich, Baslerstrasse          7.000000
Zürich, Bristenstrasse         7.000000
Zürich, Bucheggplatz          16.000000
Zürich, Escher-Wyss-Platz     17.833333
Zürich, Flurstrasse            8.833333
Zürich, Hardplatz             28.666667
Zürich, Herdernstrasse         0.000000
Zürich, Hubertus              18.500000
Zürich, Kappeli                5.833333
Zürich, Letzibach              4.000000
Zürich, Letzipark              2.000000
Zürich, Letzipark West         5.666667
Zürich, Milchbuck             17.000000
Zürich, Rosengartenstrasse    19.666667
Zürich, SBB-Werkstätte         2.000000
Zürich, Schiffbau             19.500000
Name: itcs_numberOfPassengers, dtype: float64

In [16]:
precise.head()

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2021-05-18T04:23:09Z,1621311789,3671.649,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,1,1,292.65,247500.0,0.0
1,2021-05-18T04:23:10Z,1621311790,3088.058,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,1,1,292.65,247500.0,0.0
2,2021-05-18T04:23:11Z,1621311791,3681.692,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,1,1,293.15,247500.0,0.0
3,2021-05-18T04:23:12Z,1621311792,3575.137,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,1,1,293.15,247500.0,0.0
4,2021-05-18T04:23:13Z,1621311793,3324.344,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,1,1,293.15,247500.0,0.0


In [17]:
precise.dtypes

time_iso                       object
time_unix                       int64
electric_powerDemand          float64
gnss_altitude                 float64
gnss_course                   float64
gnss_latitude                 float64
gnss_longitude                float64
itcs_busRoute                  object
itcs_numberOfPassengers       float64
itcs_stopName                  object
odometry_articulationAngle    float64
odometry_steeringAngle        float64
odometry_vehicleSpeed         float64
odometry_wheelSpeed_fl        float64
odometry_wheelSpeed_fr        float64
odometry_wheelSpeed_ml        float64
odometry_wheelSpeed_mr        float64
odometry_wheelSpeed_rl        float64
odometry_wheelSpeed_rr        float64
status_doorIsOpen               int64
status_gridIsAvailable          int64
status_haltBrakeIsActive        int64
status_parkBrakeIsActive        int64
temperature_ambient           float64
traction_brakePressure        float64
traction_tractionForce        float64
dtype: objec

In [18]:
precise2 = pd.read_csv('/Users/andreacardinali/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano-Bicocca/focs/Exam project/Project_cs23/ZTBus_compressed/B183_2021-05-18_13-58-55_2021-05-18_18-10-34.csv')
precise2.groupby('itcs_stopName')['itcs_numberOfPassengers'].mean()

itcs_stopName
-                                NaN
Zürich, Albisrank             15.625
Zürich, Albisriederplatz      20.875
Zürich, Altes Krematorium     19.250
Zürich, Bahnhof Altstetten     5.750
Zürich, Bahnhof Hardbrücke    22.375
Zürich, Baslerstrasse          6.500
Zürich, Bristenstrasse         6.500
Zürich, Bucheggplatz          18.625
Zürich, Escher-Wyss-Platz     23.000
Zürich, Flurstrasse           15.000
Zürich, Hardplatz             27.375
Zürich, Herdernstrasse         0.000
Zürich, Hubertus              18.000
Zürich, Kappeli               13.125
Zürich, Letzibach             14.400
Zürich, Letzipark             12.000
Zürich, Letzipark West        11.125
Zürich, Milchbuck             12.000
Zürich, Rosengartenstrasse    22.750
Zürich, SBB-Werkstätte        11.000
Zürich, Schiffbau             21.125
Name: itcs_numberOfPassengers, dtype: float64

In [19]:
precise3 = pd.read_csv('/Users/andreacardinali/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano-Bicocca/focs/Exam project/Project_cs23/ZTBus_compressed/B208_2021-06-07_03-54-28_2021-06-07_18-46-25.csv')
precise3.groupby('itcs_stopName')['itcs_numberOfPassengers'].mean()

itcs_stopName
-                                   NaN
Zürich, Albisrank             15.275862
Zürich, Albisriederplatz      22.551724
Zürich, Altes Krematorium     20.000000
Zürich, Bahnhof Altstetten     6.785714
Zürich, Bahnhof Hardbrücke    24.517241
Zürich, Baslerstrasse          6.714286
Zürich, Bristenstrasse         6.714286
Zürich, Bucheggplatz          18.517241
Zürich, Escher-Wyss-Platz     21.137931
Zürich, Flurstrasse           11.965517
Zürich, Hardplatz             27.068966
Zürich, Herdernstrasse         0.000000
Zürich, Hubertus              18.137931
Zürich, Kappeli                9.413793
Zürich, Letzibach              8.312500
Zürich, Letzipark              2.000000
Zürich, Letzipark West         8.413793
Zürich, Milchbuck             16.266667
Zürich, Rosengartenstrasse    21.344828
Zürich, SBB-Werkstätte         2.000000
Zürich, Schiffbau             20.862069
Name: itcs_numberOfPassengers, dtype: float64

In [20]:
precise.columns

Index(['time_iso', 'time_unix', 'electric_powerDemand', 'gnss_altitude',
       'gnss_course', 'gnss_latitude', 'gnss_longitude', 'itcs_busRoute',
       'itcs_numberOfPassengers', 'itcs_stopName',
       'odometry_articulationAngle', 'odometry_steeringAngle',
       'odometry_vehicleSpeed', 'odometry_wheelSpeed_fl',
       'odometry_wheelSpeed_fr', 'odometry_wheelSpeed_ml',
       'odometry_wheelSpeed_mr', 'odometry_wheelSpeed_rl',
       'odometry_wheelSpeed_rr', 'status_doorIsOpen', 'status_gridIsAvailable',
       'status_haltBrakeIsActive', 'status_parkBrakeIsActive',
       'temperature_ambient', 'traction_brakePressure',
       'traction_tractionForce'],
      dtype='object')

In [ ]:
diz ={}
dtype_map = {'itcs_numberOfPassengers': float,
             'itcs_stopName':str}
for csv_file in csv_files:
    diz[csv_file] = pd.read_csv(f'/Users/andreacardinali/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano-Bicocca/focs/Exam project/Project_cs23/ZTBus_compressed/{csv_file}',encoding='utf-8',na_values='-', usecols=['itcs_numberOfPassengers','itcs_stopName'], dtype=dtype_map)

In [29]:
newdir = '/Users/andreacardinali/Downloads/ZTBus_samples'

In [30]:
dic = {}
for csv_file in newdir:
    path = pd.read_csv(csv_file)
    dd = pd.read_csv(path)
    dic[csv_file] = dd['itcs_numerOfPassengers','itcs_stopName']


IsADirectoryError: [Errno 21] Is a directory: '/'

In [ ]:
ll = pd.DataFrame()
for csv_file in csv_files:
    path = os.path.join(directory, csv_file)
    dd = pd.read_csv(path, skip_blank_lines=True)
    hs = dd.groupby('itcs_stopName')['itcs_numberOfPassengers'].mean()
    pd.concat(ll, hs)
    


In [ ]:
stazioni_temp = {}
stazioni_fin = {}
 
#per ogni file in ogni cartella, ricorsivamente se necessario
#va cambiato il percorso
for dname, dirs, files in os.walk(directory):
    for file in files:
    
        # potrebbe fallire qualcosa, usiamo un try-except
        
            #leggo il file
            with open(os.path.join(dname,file),'r') as file_corrente:
            
                # l'header non ci serve
                header = next(file_corrente)
                
                # trattiamo il resto come un csv
                for riga in csv.reader(file_corrente):
                
                    # filtriamo i dati
                    
                        # stazione non valida
                        if riga[10] == "-" and riga[9]=="-":
                            continue
                            
                        # è la prima volta che vedo la stazione?
                        elif riga[10] not in stazioni_temp.keys():
                            
                            #inizilializzazione
                            stazioni_temp[riga[10]] = []
       
                        #se il numero dei passeggeri non è valido lanceremo un errore che stiamo intercettando
                        #aggiungo il valore alla relativa stazione
                        #non sto trattando il caso in cui il numero sia negativo, non dovrebbe esserlo quindi glisso
                        stazioni_temp[riga[10]].append(riga[9])
   
#ho i dati, ora calcolo le medie
#per ogni chiave e valore nel dizionario
for key, value in stazioni_temp:
    
    #calcolo la media, che è uguale alla somma di tutti gli elementi diviso il numero di tali elementi
    stazioni_fin[key]=sum(value)/len(value)
    
#stampo i risultati
for key, value in stazioni_fin:
    print("La stazione",key,"ha in media",value,"persone")

### 6. For each station, determine the buses that have stopped there at least once.

### 7. For each station, determine the buses that have stopped there at least ten times.

### 8. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance. 

In [ ]:
df2 = df.groupby(['busNumber','busRoute'])[['drivenDistance','energyConsumption']].sum()

In [ ]:
df2.reset_index(inplace=True)

In [ ]:
df2['ratio'] = df2['energyConsumption']/df2['drivenDistance']

In [ ]:
df2

,busNumber,busRoute,drivenDistance,energyConsumption,ratio
0,183,31.0,2279952.00,1.387782e+10,6086.891785
1,183,32.0,2256540.10,1.393320e+10,6174.585331
2,183,33.0,29510481.41,1.761996e+11,5970.747507
3,183,46.0,18224657.94,1.026237e+11,5631.035959
4,183,72.0,23637238.45,1.394368e+11,5899.029643
5,183,83.0,51249813.70,2.995540e+11,5844.977154
6,208,31.0,1046306.68,5.535865e+09,5290.862713
7,208,32.0,2337064.21,1.283340e+10,5491.247671
8,208,33.0,6161892.39,3.474842e+10,5639.244537
9,208,46.0,4186493.70,2.337366e+10,5583.110707


### 9. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.

In [ ]:
groupmax = df2.groupby('busRoute')[['busNumber','ratio']].max()
groupmin = df2.groupby('busRoute')[['busNumber','ratio']].min()
df3 = pd.merge(groupmax, groupmin, on = 'busRoute')

In [ ]:
df3.reset_index(inplace=True)

In [ ]:
df3['difference'] = df3['ratio_x'] - df3['ratio_y']
df3.head()

,busRoute,busNumber_x,ratio_x,busNumber_y,ratio_y,difference
0,31.0,208,6086.891785,183,5290.862713,796.029072
1,32.0,208,6174.585331,183,5491.247671,683.337660
2,33.0,208,5970.747507,183,5639.244537,331.502969
3,46.0,208,5631.035959,183,5583.110707,47.925252
4,72.0,208,5899.029643,183,5410.318878,488.710765


### 10. Find the bus maximizing the difference computed in the previous point.

In [ ]:
df3.loc[df3['difference'].idxmax()]

busRoute         31.000000
busNumber_x     208.000000
ratio_x        6086.891785
busNumber_y     183.000000
ratio_y        5290.862713
difference      796.029072
Name: 0, dtype: float64

### 11. Extract the rows of the details such that the `gnss_altitude` differs from the value in the preceding row. Store also the difference in the variable `altitude_variation`.

### 12. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of `altitude_variation`.

### 13.  For each month of the year, compute the average ambient temperature

### 14. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.

### 15. For each pair of stops that are consecutive in some trip, compute the average speed achieved when going from the first to the second stop.